In [1]:
import torch
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import time
import torch.nn as nn
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from Model import ForexTransformer
from sklearn.model_selection import train_test_split
from twelvedata import TDClient 

In [2]:
seq_length = 20
num_layers = 4
num_heads = 8
FILE_PATH = "Dataset\EURUSD\EURUSD_M30_with_features+label3.csv"
API_KEY = 'e6b3ac6147a04e34bca8d5f1011b728c'
features = ['SMA100', 'RSI14', 'Close']

<>:4: SyntaxWarning: invalid escape sequence '\E'
<>:4: SyntaxWarning: invalid escape sequence '\E'
C:\Users\Stephen\AppData\Local\Temp\ipykernel_26148\3306518988.py:4: SyntaxWarning: invalid escape sequence '\E'
  FILE_PATH = "Dataset\EURUSD\EURUSD_M30_with_features+label3.csv"


In [3]:
td = TDClient(apikey=API_KEY)
data = pd.read_csv(FILE_PATH)
# Select features and labels
labels = 'signal'
X = data[features].values
y = data[labels].values
# Normalize the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
# Convert labels to integers
num_classes = len(set(y))
y = torch.tensor(y, dtype=torch.long)
# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=13)
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
def create_sequences(data, labels, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(data) - seq_length + 1):
        X_seq.append(data[i:i + seq_length])
        y_seq.append(labels[i + seq_length - 1])  # Directly append the label without .item()
    return torch.stack(X_seq), torch.tensor(y_seq, dtype=torch.long)
X_train_seq, y_train_seq = create_sequences(X_train_tensor, y_train_tensor, seq_length)
X_val_seq, y_val_seq = create_sequences(X_val_tensor, y_val_tensor, seq_length)

C:\Users\Stephen\AppData\Local\Temp\ipykernel_26148\327436413.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train, dtype=torch.long)
C:\Users\Stephen\AppData\Local\Temp\ipykernel_26148\327436413.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_val_tensor = torch.tensor(y_val, dtype=torch.long)


In [4]:
# Load the trained model
MODEL_PATH = "D:/Programing/AI Trader/Model/transformerModelv.2.0_layers-heads 4-8_Val-Loss 0.9002, Val-Accuracy 0.5699_at 20241216-212047.model"
model = ForexTransformer(input_dim=X_train_seq.shape[2], seq_length=seq_length, num_heads=num_heads, num_layers=num_layers, output_dim=num_classes).to('cuda')
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

c:\Program Files\Python312\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
C:\Users\Stephen\AppData\Local\Temp\ipykernel_26148\790591655.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.a

ForexTransformer(
  (embedding): Linear(in_features=3, out_features=256, bias=True)
  (pos_encoder): PositionalEncoding()
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linear(in_features=256, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=256, bias=True)
        (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (dropout): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=5120, out_features=4, bias=True)
)

In [5]:
# Define a function to preprocess real-time data
def preprocess_realtime_data(data, features=['sma', 'rsi', 'close'], seq_length=seq_length):
    # data = data.drop()
    # Select and normalize features
    data = data[features]
    data_scaled = scaler.fit_transform(data)

    # Prepare sequence for the model
    if len(data_scaled) >= seq_length:
        input_seq = data_scaled[-seq_length:]
    else:
        raise ValueError("Not enough data for the required sequence length.")

    return torch.tensor(input_seq, dtype=torch.float32).unsqueeze(0).to('cuda')



In [6]:
# Mock function to simulate fetching real-time data
def fetch_realtime_data(interval="30min"):

    ts = td.time_series(symbol="EUR/USD", interval=interval, outputsize=seq_length).with_sma(time_period=100).with_rsi(time_period=14).as_pandas()
    
    return ts


In [7]:
mint = 5

In [8]:
# fetch_realtime_data(f"{mint}min")

In [9]:
# Define a function to make predictions in real time
def make_realtime_prediction(model):
    while True:
        # Fetch real-time data
        realtime_data = fetch_realtime_data(f"{mint}min")

        try:
            # Preprocess the data
            input_tensor = preprocess_realtime_data(realtime_data)

            # Make a prediction
            with torch.no_grad():
                output = model(input_tensor)

            # Interpret the output
            condition = torch.argmax(output, axis=1).item()
            conditions_map = {0: "Buy", 1: "Sell", 2: "Hold", 3: "Nothing"}
            prediction = conditions_map[condition]

            # Print the prediction
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"Real-time Prediction: {prediction} | {timestamp}")

        except ValueError as e:
            print(e)

        # Pause for a short time before fetching new data
        time.sleep(mint*60)  # Adjust the delay as needed

In [ ]:
# Run the real-time prediction function
make_realtime_prediction(model)

Real-time Prediction: Buy | 2024-12-17 10:40:34
Real-time Prediction: Buy | 2024-12-17 10:45:35
Real-time Prediction: Buy | 2024-12-17 10:50:35
Real-time Prediction: Buy | 2024-12-17 10:55:36
Real-time Prediction: Buy | 2024-12-17 11:00:36
Real-time Prediction: Buy | 2024-12-17 11:05:38
Real-time Prediction: Buy | 2024-12-17 11:10:39
Real-time Prediction: Buy | 2024-12-17 11:15:40
Real-time Prediction: Buy | 2024-12-17 11:20:42
Real-time Prediction: Buy | 2024-12-17 11:25:44
Real-time Prediction: Hold | 2024-12-17 11:30:45
Real-time Prediction: Sell | 2024-12-17 11:35:45
Real-time Prediction: Buy | 2024-12-17 11:40:45
Real-time Prediction: Buy | 2024-12-17 11:45:47
Real-time Prediction: Sell | 2024-12-17 11:50:47
Real-time Prediction: Buy | 2024-12-17 11:55:47
Real-time Prediction: Buy | 2024-12-17 12:00:48
Real-time Prediction: Buy | 2024-12-17 12:05:48
Real-time Prediction: Buy | 2024-12-17 12:10:49
Real-time Prediction: Buy | 2024-12-17 12:15:49
Real-time Prediction: Buy | 2024-12-1